In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [14, 15, 17]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.078963279724121, Final Batch Loss: 2.034856081008911
Epoch 2, Loss: 4.058208465576172, Final Batch Loss: 2.025503396987915
Epoch 3, Loss: 4.041425466537476, Final Batch Loss: 2.0212910175323486
Epoch 4, Loss: 4.024147272109985, Final Batch Loss: 2.012317180633545
Epoch 5, Loss: 4.001115322113037, Final Batch Loss: 1.9981250762939453
Epoch 6, Loss: 3.9739232063293457, Final Batch Loss: 1.9851902723312378
Epoch 7, Loss: 3.959748864173889, Final Batch Loss: 1.9802050590515137
Epoch 8, Loss: 3.929610252380371, Final Batch Loss: 1.957871675491333
Epoch 9, Loss: 3.9117616415023804, Final Batch Loss: 1.9478113651275635
Epoch 10, Loss: 3.883876323699951, Final Batch Loss: 1.9407999515533447
Epoch 11, Loss: 3.8524307012557983, Final Batch Loss: 1.9215139150619507
Epoch 12, Loss: 3.8124618530273438, Final Batch Loss: 1.9053488969802856
Epoch 13, Loss: 3.77833092212677, Final Batch Loss: 1.8823391199111938
Epoch 14, Loss: 3.729729175567627, Final Batch Loss: 1.8594715595245361
Ep

Epoch 113, Loss: 0.18484464287757874, Final Batch Loss: 0.09799067676067352
Epoch 114, Loss: 0.19096579402685165, Final Batch Loss: 0.11104916036128998
Epoch 115, Loss: 0.16206420958042145, Final Batch Loss: 0.0718717947602272
Epoch 116, Loss: 0.12363890931010246, Final Batch Loss: 0.06573349237442017
Epoch 117, Loss: 0.1216634102165699, Final Batch Loss: 0.047695714980363846
Epoch 118, Loss: 0.15961245447397232, Final Batch Loss: 0.10527409613132477
Epoch 119, Loss: 0.12272849306464195, Final Batch Loss: 0.07771702110767365
Epoch 120, Loss: 0.1786285862326622, Final Batch Loss: 0.09474918246269226
Epoch 121, Loss: 0.13250815868377686, Final Batch Loss: 0.06909279525279999
Epoch 122, Loss: 0.11212491244077682, Final Batch Loss: 0.06555783748626709
Epoch 123, Loss: 0.17154225707054138, Final Batch Loss: 0.07543251663446426
Epoch 124, Loss: 0.11735688149929047, Final Batch Loss: 0.038710758090019226
Epoch 125, Loss: 0.13672852143645287, Final Batch Loss: 0.07814313471317291
Epoch 126, Lo

Epoch 245, Loss: 0.029714398086071014, Final Batch Loss: 0.012694869190454483
Epoch 246, Loss: 0.06191334128379822, Final Batch Loss: 0.035954784601926804
Epoch 247, Loss: 0.028933795168995857, Final Batch Loss: 0.015721814706921577
Epoch 248, Loss: 0.016712781973183155, Final Batch Loss: 0.008124716579914093
Epoch 249, Loss: 0.060019451193511486, Final Batch Loss: 0.047722239047288895
Epoch 250, Loss: 0.017103967256844044, Final Batch Loss: 0.008770931512117386
Epoch 251, Loss: 0.0617985874414444, Final Batch Loss: 0.03789651393890381
Epoch 252, Loss: 0.012831022497266531, Final Batch Loss: 0.007028212770819664
Epoch 253, Loss: 0.031006078235805035, Final Batch Loss: 0.012936300598084927
Epoch 254, Loss: 0.02613584604114294, Final Batch Loss: 0.010007248260080814
Epoch 255, Loss: 0.030788345262408257, Final Batch Loss: 0.013643166050314903
Epoch 256, Loss: 0.05008656159043312, Final Batch Loss: 0.04005475714802742
Epoch 257, Loss: 0.017091648653149605, Final Batch Loss: 0.006537325680

Epoch 371, Loss: 0.019866991322487593, Final Batch Loss: 0.014954989776015282
Epoch 372, Loss: 0.02011938439682126, Final Batch Loss: 0.01565307006239891
Epoch 373, Loss: 0.018858415074646473, Final Batch Loss: 0.007356660440564156
Epoch 374, Loss: 0.029019074514508247, Final Batch Loss: 0.008674843236804008
Epoch 375, Loss: 0.011430058162659407, Final Batch Loss: 0.004163600504398346
Epoch 376, Loss: 0.05316422414034605, Final Batch Loss: 0.041384488344192505
Epoch 377, Loss: 0.014655235689133406, Final Batch Loss: 0.005229210015386343
Epoch 378, Loss: 0.013123358832672238, Final Batch Loss: 0.003223507897928357
Epoch 379, Loss: 0.018873596098273993, Final Batch Loss: 0.0048141260631382465
Epoch 380, Loss: 0.028731065802276134, Final Batch Loss: 0.006064972840249538
Epoch 381, Loss: 0.02498052967712283, Final Batch Loss: 0.003790860529989004
Epoch 382, Loss: 0.010132243391126394, Final Batch Loss: 0.0055182212963700294
Epoch 383, Loss: 0.01872100541368127, Final Batch Loss: 0.00579248

Epoch 478, Loss: 0.006162018631584942, Final Batch Loss: 0.0008930709445849061
Epoch 479, Loss: 0.010198021307587624, Final Batch Loss: 0.0010079499334096909
Epoch 480, Loss: 0.009804020170122385, Final Batch Loss: 0.004294665530323982
Epoch 481, Loss: 0.015328339766710997, Final Batch Loss: 0.010236617177724838
Epoch 482, Loss: 0.036054931581020355, Final Batch Loss: 0.027095763012766838
Epoch 483, Loss: 0.011273069307208061, Final Batch Loss: 0.001063324511051178
Epoch 484, Loss: 0.019649074412882328, Final Batch Loss: 0.009957073256373405
Epoch 485, Loss: 0.008751828223466873, Final Batch Loss: 0.002448217011988163
Epoch 486, Loss: 0.004066147259436548, Final Batch Loss: 0.0026690508238971233
Epoch 487, Loss: 0.031048834789544344, Final Batch Loss: 0.025006135925650597
Epoch 488, Loss: 0.009156413841992617, Final Batch Loss: 0.003920154646039009
Epoch 489, Loss: 0.01411270466633141, Final Batch Loss: 0.002618562662974
Epoch 490, Loss: 0.013701556948944926, Final Batch Loss: 0.003443

...

Epoch 4472, Loss: 0.000653079041512683, Final Batch Loss: 0.00016356594278477132
Epoch 4473, Loss: 0.00010810681669681799, Final Batch Loss: 8.382921078009531e-05
Epoch 4474, Loss: 0.0003701043953583394, Final Batch Loss: 4.994564619664743e-07
Epoch 4475, Loss: 4.28727778398752e-05, Final Batch Loss: 2.8163428851257777e-06
Epoch 4476, Loss: 1.3457532531901961e-05, Final Batch Loss: 7.952044143166859e-06
Epoch 4477, Loss: 0.0003176086629537167, Final Batch Loss: 1.3345839761313982e-05
Epoch 4478, Loss: 3.698356613313081e-05, Final Batch Loss: 3.023071985808201e-05
Epoch 4479, Loss: 0.0014006809656166297, Final Batch Loss: 0.001393970218487084
Epoch 4480, Loss: 5.063143862571451e-05, Final Batch Loss: 4.272882051736815e-06
Epoch 4481, Loss: 0.000788130786986585, Final Batch Loss: 1.5426089703396428e-06
Epoch 4482, Loss: 0.0015039337795315078, Final Batch Loss: 0.0014743591891601682
Epoch 4483, Loss: 1.4223209518604563e-05, Final Batch Loss: 2.922571411545505e-06
Epoch 4484, Loss: 3.02876

Epoch 4582, Loss: 8.911903569241986e-05, Final Batch Loss: 2.676098665688187e-05
Epoch 4583, Loss: 1.6756398281358997e-05, Final Batch Loss: 1.1328509117447538e-06
Epoch 4584, Loss: 0.0008969204823188193, Final Batch Loss: 0.0008955320226959884
Epoch 4585, Loss: 4.783058864177292e-05, Final Batch Loss: 4.6608009142801166e-05
Epoch 4586, Loss: 4.931017798526227e-07, Final Batch Loss: 1.583651823011678e-07
Epoch 4587, Loss: 1.0211478638666449e-05, Final Batch Loss: 6.834349278506124e-06
Epoch 4588, Loss: 0.00025002974985000037, Final Batch Loss: 0.00024752612807787955
Epoch 4589, Loss: 3.062097670181174e-05, Final Batch Loss: 2.1057343246866367e-07
Epoch 4590, Loss: 5.476728379960605e-05, Final Batch Loss: 1.6010504566565942e-07
Epoch 4591, Loss: 6.778050453704054e-05, Final Batch Loss: 1.654870061429392e-06
Epoch 4592, Loss: 0.000610807497650967, Final Batch Loss: 0.0005972756189294159
Epoch 4593, Loss: 5.055287715549639e-06, Final Batch Loss: 3.77168566956243e-06
Epoch 4594, Loss: 5.51

Epoch 4692, Loss: 8.653160648464109e-06, Final Batch Loss: 8.387685284105828e-07
Epoch 4693, Loss: 1.276222121759929e-06, Final Batch Loss: 5.586160796156037e-07
Epoch 4694, Loss: 7.074097266013268e-05, Final Batch Loss: 1.746583620843012e-05
Epoch 4695, Loss: 2.5372762593178777e-06, Final Batch Loss: 6.229997779882979e-07
Epoch 4696, Loss: 1.7610920394872664e-05, Final Batch Loss: 2.7135381515108747e-06
Epoch 4697, Loss: 3.008610008237156e-05, Final Batch Loss: 1.7105784309023875e-06
Epoch 4698, Loss: 9.972872794605792e-05, Final Batch Loss: 8.197288843803108e-05
Epoch 4699, Loss: 0.0005887204206374008, Final Batch Loss: 3.552680209395476e-05
Epoch 4700, Loss: 0.000326004814269254, Final Batch Loss: 0.000294949219096452
Epoch 4701, Loss: 0.048806556113504485, Final Batch Loss: 0.04879896715283394
Epoch 4702, Loss: 3.2127223903444246e-06, Final Batch Loss: 2.2047261154511943e-06
Epoch 4703, Loss: 5.1121234719175845e-05, Final Batch Loss: 7.5831630965694785e-06
Epoch 4704, Loss: 2.67386

Epoch 4813, Loss: 0.0011406005924072815, Final Batch Loss: 2.517882785468828e-05
Epoch 4814, Loss: 0.00013313742238096893, Final Batch Loss: 5.147887713974342e-05
Epoch 4815, Loss: 0.0002195140750700375, Final Batch Loss: 0.00021148010273464024
Epoch 4816, Loss: 6.497899539681384e-05, Final Batch Loss: 6.339801075228024e-06
Epoch 4817, Loss: 0.0001307498496316839, Final Batch Loss: 4.4086042180424556e-05
Epoch 4818, Loss: 0.00010362918328610249, Final Batch Loss: 1.8018061382463202e-05
Epoch 4819, Loss: 0.00016446437439299189, Final Batch Loss: 0.00013259629486128688
Epoch 4820, Loss: 0.0005192354146856815, Final Batch Loss: 0.0004937630146741867
Epoch 4821, Loss: 0.00022234179050428793, Final Batch Loss: 4.857984458794817e-05
Epoch 4822, Loss: 2.414889786450658e-05, Final Batch Loss: 1.3260505511425436e-06
Epoch 4823, Loss: 0.0010803394543472677, Final Batch Loss: 0.0009688524878583848
Epoch 4824, Loss: 0.0007390278660750482, Final Batch Loss: 0.0007127961725927889
Epoch 4825, Loss: 0

Epoch 4933, Loss: 2.5781349449971458e-05, Final Batch Loss: 2.10179241548758e-05
Epoch 4934, Loss: 7.520129486238147e-06, Final Batch Loss: 6.756724815204507e-06
Epoch 4935, Loss: 1.1839125136248185e-05, Final Batch Loss: 9.47322496358538e-06
Epoch 4936, Loss: 3.7012475786468713e-06, Final Batch Loss: 1.4365346032718662e-06
Epoch 4937, Loss: 1.9602007341745775e-05, Final Batch Loss: 9.367006896354724e-06
Epoch 4938, Loss: 4.208368318359135e-05, Final Batch Loss: 2.7108289941679686e-05
Epoch 4939, Loss: 6.153542153697344e-05, Final Batch Loss: 5.928836981183849e-05
Epoch 4940, Loss: 5.756341306550894e-05, Final Batch Loss: 1.6456075172754936e-05
Epoch 4941, Loss: 4.72591136713163e-06, Final Batch Loss: 1.8306798210687703e-06
Epoch 4942, Loss: 0.00012671362856053747, Final Batch Loss: 5.3597392252413556e-05
Epoch 4943, Loss: 0.0019339832797413692, Final Batch Loss: 0.0018774131312966347
Epoch 4944, Loss: 0.0024398429295615642, Final Batch Loss: 4.596447070071008e-06
Epoch 4945, Loss: 2.6

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[25  0  0]
 [ 0 27  0]
 [ 0  0 45]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        25
           1    1.00000   1.00000   1.00000        27
           2    1.00000   1.00000   1.00000        45

    accuracy                        1.00000        97
   macro avg    1.00000   1.00000   1.00000        97
weighted avg    1.00000   1.00000   1.00000        97



In [11]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 3)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 3)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 3) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [12]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[30  0  0]
 [ 0 30  0]
 [ 0  0 30]]
              precision    recall  f1-score   support

         0.0    1.00000   1.00000   1.00000        30
         1.0    1.00000   1.00000   1.00000        30
         2.0    1.00000   1.00000   1.00000        30

    accuracy                        1.00000        90
   macro avg    1.00000   1.00000   1.00000        90
weighted avg    1.00000   1.00000   1.00000        90



# User Classifier

In [13]:
activities = [1, 3, 4]
users = [14, 15, 17]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [14]:
for k in range(len(y)):
    if y[k] == 14:
        y[k] = 0
    elif y[k] == 15:
        y[k] = 1
    else:
        y[k] = 2

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [16]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.212775945663452, Final Batch Loss: 2.1009202003479004
Epoch 2, Loss: 4.208991289138794, Final Batch Loss: 2.1168651580810547
Epoch 3, Loss: 4.18974232673645, Final Batch Loss: 2.098130702972412
Epoch 4, Loss: 4.178011417388916, Final Batch Loss: 2.0885064601898193
Epoch 5, Loss: 4.1669018268585205, Final Batch Loss: 2.0820682048797607
Epoch 6, Loss: 4.149097681045532, Final Batch Loss: 2.071721315383911
Epoch 7, Loss: 4.126430511474609, Final Batch Loss: 2.0566389560699463
Epoch 8, Loss: 4.110490560531616, Final Batch Loss: 2.0427193641662598
Epoch 9, Loss: 4.105367183685303, Final Batch Loss: 2.0580835342407227
Epoch 10, Loss: 4.087891101837158, Final Batch Loss: 2.033804416656494
Epoch 11, Loss: 4.063032150268555, Final Batch Loss: 2.0144364833831787
Epoch 12, Loss: 4.0352044105529785, Final Batch Loss: 2.004441022872925
Epoch 13, Loss: 4.027859449386597, Final Batch Loss: 2.0145206451416016
Epoch 14, Loss: 4.0024871826171875, Final Batch Loss: 2.0002188682556152
Epo

Epoch 121, Loss: 1.5377275347709656, Final Batch Loss: 0.782069206237793
Epoch 122, Loss: 1.5177512764930725, Final Batch Loss: 0.7822418808937073
Epoch 123, Loss: 1.5593517422676086, Final Batch Loss: 0.8305290341377258
Epoch 124, Loss: 1.4834073185920715, Final Batch Loss: 0.7316708564758301
Epoch 125, Loss: 1.5139175653457642, Final Batch Loss: 0.7761777639389038
Epoch 126, Loss: 1.4916625022888184, Final Batch Loss: 0.7640619874000549
Epoch 127, Loss: 1.5680912137031555, Final Batch Loss: 0.8093515038490295
Epoch 128, Loss: 1.4780933260917664, Final Batch Loss: 0.7141973972320557
Epoch 129, Loss: 1.5635334849357605, Final Batch Loss: 0.7980458736419678
Epoch 130, Loss: 1.565445899963379, Final Batch Loss: 0.8232944011688232
Epoch 131, Loss: 1.4670230150222778, Final Batch Loss: 0.7315845489501953
Epoch 132, Loss: 1.4910864233970642, Final Batch Loss: 0.7470732927322388
Epoch 133, Loss: 1.4618062376976013, Final Batch Loss: 0.7206176519393921
Epoch 134, Loss: 1.5094990134239197, Fin

Epoch 236, Loss: 1.0951869487762451, Final Batch Loss: 0.548299252986908
Epoch 237, Loss: 1.1333884596824646, Final Batch Loss: 0.6008874177932739
Epoch 238, Loss: 1.1065813302993774, Final Batch Loss: 0.5880618691444397
Epoch 239, Loss: 1.0402279496192932, Final Batch Loss: 0.5231348872184753
Epoch 240, Loss: 1.0630711913108826, Final Batch Loss: 0.5306309461593628
Epoch 241, Loss: 0.9903439283370972, Final Batch Loss: 0.45720911026000977
Epoch 242, Loss: 1.0253254175186157, Final Batch Loss: 0.5192955136299133
Epoch 243, Loss: 1.0185168981552124, Final Batch Loss: 0.5013346672058105
Epoch 244, Loss: 0.9427862763404846, Final Batch Loss: 0.44443559646606445
Epoch 245, Loss: 1.0170587301254272, Final Batch Loss: 0.5162209272384644
Epoch 246, Loss: 1.0024062991142273, Final Batch Loss: 0.4869190454483032
Epoch 247, Loss: 0.9887283742427826, Final Batch Loss: 0.49556809663772583
Epoch 248, Loss: 0.958329975605011, Final Batch Loss: 0.4616200625896454
Epoch 249, Loss: 0.9287835955619812, 

Epoch 360, Loss: 0.6216825544834137, Final Batch Loss: 0.29556167125701904
Epoch 361, Loss: 0.6519550681114197, Final Batch Loss: 0.35062500834465027
Epoch 362, Loss: 0.6352902948856354, Final Batch Loss: 0.26095759868621826
Epoch 363, Loss: 0.5763691663742065, Final Batch Loss: 0.2627629041671753
Epoch 364, Loss: 0.611301988363266, Final Batch Loss: 0.2773798108100891
Epoch 365, Loss: 0.601038247346878, Final Batch Loss: 0.2820705771446228
Epoch 366, Loss: 0.5867957770824432, Final Batch Loss: 0.27291637659072876
Epoch 367, Loss: 0.6321539580821991, Final Batch Loss: 0.34069326519966125
Epoch 368, Loss: 0.5366683751344681, Final Batch Loss: 0.22047816216945648
Epoch 369, Loss: 0.5943033993244171, Final Batch Loss: 0.28367337584495544
Epoch 370, Loss: 0.66403529047966, Final Batch Loss: 0.32969969511032104
Epoch 371, Loss: 0.6488195657730103, Final Batch Loss: 0.314433753490448
Epoch 372, Loss: 0.6690360307693481, Final Batch Loss: 0.3579172194004059
Epoch 373, Loss: 0.5720158517360687

Epoch 486, Loss: 0.5208021402359009, Final Batch Loss: 0.28566116094589233
Epoch 487, Loss: 0.5437843501567841, Final Batch Loss: 0.24345055222511292
Epoch 488, Loss: 0.5624257922172546, Final Batch Loss: 0.33774974942207336
Epoch 489, Loss: 0.5221356302499771, Final Batch Loss: 0.29093897342681885
Epoch 490, Loss: 0.5012147575616837, Final Batch Loss: 0.20549754798412323
Epoch 491, Loss: 0.5228656232357025, Final Batch Loss: 0.27310910820961
Epoch 492, Loss: 0.5065844506025314, Final Batch Loss: 0.2687583565711975
Epoch 493, Loss: 0.6159967482089996, Final Batch Loss: 0.3778725862503052
Epoch 494, Loss: 0.45843377709388733, Final Batch Loss: 0.23520977795124054
Epoch 495, Loss: 0.541085347533226, Final Batch Loss: 0.1942162662744522
Epoch 496, Loss: 0.46951794624328613, Final Batch Loss: 0.20273825526237488
Epoch 497, Loss: 0.5078954100608826, Final Batch Loss: 0.24704325199127197
Epoch 498, Loss: 0.5250839591026306, Final Batch Loss: 0.257376492023468
Epoch 499, Loss: 0.5269320309162

...

Epoch 4424, Loss: 0.01714492030441761, Final Batch Loss: 0.011080662719905376
Epoch 4425, Loss: 0.018720888532698154, Final Batch Loss: 0.011023454368114471
Epoch 4426, Loss: 0.08826787443831563, Final Batch Loss: 0.08107277005910873
Epoch 4427, Loss: 0.0969167910516262, Final Batch Loss: 0.024514567106962204
Epoch 4428, Loss: 0.03688941616564989, Final Batch Loss: 0.022253679111599922
Epoch 4429, Loss: 0.01889229565858841, Final Batch Loss: 0.009999857284128666
Epoch 4430, Loss: 0.0150603661313653, Final Batch Loss: 0.009270657785236835
Epoch 4431, Loss: 0.012554254615679383, Final Batch Loss: 0.0017798955086618662
Epoch 4432, Loss: 0.0339425140991807, Final Batch Loss: 0.006076029501855373
Epoch 4433, Loss: 0.03046681359410286, Final Batch Loss: 0.024784468114376068
Epoch 4434, Loss: 0.03392417263239622, Final Batch Loss: 0.011535721831023693
Epoch 4435, Loss: 0.052069809287786484, Final Batch Loss: 0.036513835191726685
Epoch 4436, Loss: 0.02156996075063944, Final Batch Loss: 0.00442

Epoch 4532, Loss: 0.05289357528090477, Final Batch Loss: 0.047624099999666214
Epoch 4533, Loss: 0.015822704881429672, Final Batch Loss: 0.008719358593225479
Epoch 4534, Loss: 0.05821964889764786, Final Batch Loss: 0.01573287695646286
Epoch 4535, Loss: 0.017804550705477595, Final Batch Loss: 0.0035056553315371275
Epoch 4536, Loss: 0.00941725680604577, Final Batch Loss: 0.0019697826355695724
Epoch 4537, Loss: 0.017646665568463504, Final Batch Loss: 0.0017968666506931186
Epoch 4538, Loss: 0.04209689702838659, Final Batch Loss: 0.013085554353892803
Epoch 4539, Loss: 0.06241531856358051, Final Batch Loss: 0.035661689937114716
Epoch 4540, Loss: 0.03146099764853716, Final Batch Loss: 0.024527469649910927
Epoch 4541, Loss: 0.05493887048214674, Final Batch Loss: 0.049650222063064575
Epoch 4542, Loss: 0.01723745046183467, Final Batch Loss: 0.007036540191620588
Epoch 4543, Loss: 0.05664627440273762, Final Batch Loss: 0.015140006318688393
Epoch 4544, Loss: 0.034146467223763466, Final Batch Loss: 0

Epoch 4642, Loss: 0.039322853088378906, Final Batch Loss: 0.015670454129576683
Epoch 4643, Loss: 0.06799942627549171, Final Batch Loss: 0.03137368708848953
Epoch 4644, Loss: 0.08481663465499878, Final Batch Loss: 0.05948471277952194
Epoch 4645, Loss: 0.18606941401958466, Final Batch Loss: 0.1413642317056656
Epoch 4646, Loss: 0.08562006987631321, Final Batch Loss: 0.026797166094183922
Epoch 4647, Loss: 0.07601959817111492, Final Batch Loss: 0.026083694770932198
Epoch 4648, Loss: 0.09000075608491898, Final Batch Loss: 0.05759857967495918
Epoch 4649, Loss: 0.02062493795529008, Final Batch Loss: 0.004882868845015764
Epoch 4650, Loss: 0.11732861027121544, Final Batch Loss: 0.07852551341056824
Epoch 4651, Loss: 0.014557241927832365, Final Batch Loss: 0.005219979677349329
Epoch 4652, Loss: 0.03810556326061487, Final Batch Loss: 0.030074981972575188
Epoch 4653, Loss: 0.027849999256432056, Final Batch Loss: 0.006794902496039867
Epoch 4654, Loss: 0.05948106665164232, Final Batch Loss: 0.01274969

Epoch 4768, Loss: 0.03077455423772335, Final Batch Loss: 0.022547075524926186
Epoch 4769, Loss: 0.030847984366118908, Final Batch Loss: 0.014541669748723507
Epoch 4770, Loss: 0.04379897937178612, Final Batch Loss: 0.025616761296987534
Epoch 4771, Loss: 0.021705929189920425, Final Batch Loss: 0.016993040218949318
Epoch 4772, Loss: 0.014528241474181414, Final Batch Loss: 0.005486680660396814
Epoch 4773, Loss: 0.09479454159736633, Final Batch Loss: 0.027986422181129456
Epoch 4774, Loss: 0.023196297697722912, Final Batch Loss: 0.015719609335064888
Epoch 4775, Loss: 0.020603445824235678, Final Batch Loss: 0.015402374789118767
Epoch 4776, Loss: 0.013200992718338966, Final Batch Loss: 0.00805641245096922
Epoch 4777, Loss: 0.04248631373047829, Final Batch Loss: 0.0304876696318388
Epoch 4778, Loss: 0.010518923169001937, Final Batch Loss: 0.00688367011025548
Epoch 4779, Loss: 0.051643332466483116, Final Batch Loss: 0.008291585370898247
Epoch 4780, Loss: 0.01144018943887204, Final Batch Loss: 0.0

Epoch 4897, Loss: 0.05837918957695365, Final Batch Loss: 0.0019720769487321377
Epoch 4898, Loss: 0.012577638030052185, Final Batch Loss: 0.005253720562905073
Epoch 4899, Loss: 0.12098958063870668, Final Batch Loss: 0.11321906745433807
Epoch 4900, Loss: 0.04174316441640258, Final Batch Loss: 0.03504155948758125
Epoch 4901, Loss: 0.05952721042558551, Final Batch Loss: 0.006385821383446455
Epoch 4902, Loss: 0.1499871639534831, Final Batch Loss: 0.011945920996367931
Epoch 4903, Loss: 0.031716194935142994, Final Batch Loss: 0.005990779958665371
Epoch 4904, Loss: 0.027270644903182983, Final Batch Loss: 0.006180375814437866
Epoch 4905, Loss: 0.07112362515181303, Final Batch Loss: 0.06095818802714348
Epoch 4906, Loss: 0.010146853281185031, Final Batch Loss: 0.0019894593860954046
Epoch 4907, Loss: 0.12338182143867016, Final Batch Loss: 0.11562670022249222
Epoch 4908, Loss: 0.05043829791247845, Final Batch Loss: 0.011980211362242699
Epoch 4909, Loss: 0.04227781854569912, Final Batch Loss: 0.0180

In [17]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[35  0  0]
 [ 0 31  0]
 [ 0  0 31]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        35
           1    1.00000   1.00000   1.00000        31
           2    1.00000   1.00000   1.00000        31

    accuracy                        1.00000        97
   macro avg    1.00000   1.00000   1.00000        97
weighted avg    1.00000   1.00000   1.00000        97



In [18]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples
fake_labels = np.asarray(fake_labels)

In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[25  0  5]
 [ 4 21  5]
 [ 2  1 27]]
              precision    recall  f1-score   support

           0    0.80645   0.83333   0.81967        30
           1    0.95455   0.70000   0.80769        30
           2    0.72973   0.90000   0.80597        30

    accuracy                        0.81111        90
   macro avg    0.83024   0.81111   0.81111        90
weighted avg    0.83024   0.81111   0.81111        90

